In [ ]:
#| default_exp utils

# Utilities
> Various utilities

In [ ]:
#| export
from netCDF4 import Dataset
from fastcore.test import test_eq
from typing import Dict

## Validation

In [ ]:
#| export
def has_valid_varname(
    var_names:Dict, # Look up table associating dataset variable names to standardized one  
    cdl_path:str, # Path to MARIS CDL file (point of truth)
):
    "Check that proposed variable names are in MARIS CDL"
    has_valid = True
    with Dataset(cdl_path) as nc:
        for name in var_names.values():
            if name not in nc.variables.keys():
                has_valid = False
                print(f'"{name}" variable name not found in MARIS CDL')
    
    return has_valid

In [ ]:
VARNAMES = {
    'Sampling coordinate North latitude (Decimal)': 'lat',
    'Sampling coordinate East longitude (Decimal)': 'lon'
}

test_eq(has_valid_varname(VARNAMES, './files/nc/maris-cdl.nc'), True)

In [ ]:
VARNAMES = {
    '140Ba radioactivity concentration (Bq/L)': 'ba140_invalid',
    '140Ba detection limit (Bq/L)': 'ba140_dl'}

test_eq(has_valid_varname(VARNAMES), False)

"ba140_invalid" variable name not found in MARIS CDL
